# 把框选出的单词单独保存出来

In [1]:
import cv2
import os
import numpy as np

In [7]:
def extract_red_boxes(image):
    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the red color in the HSV space
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for the red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours, mask

def resize_image(image, size=(128, 128)):
    return cv2.resize(image, size)

def crop_words(input_folder, crop_folder):
    if not os.path.exists(crop_folder):
        os.makedirs(crop_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for img_file in image_files:
        img_path = os.path.join(input_folder, img_file)
        image = cv2.imread(img_path)
        
        contours, _ = extract_red_boxes(image)
        
        for i, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            word_crop = image[y:y+h, x:x+w]
            
            crop_path = os.path.join(crop_folder, f"{os.path.splitext(img_file)[0]}_word_{i}.png")
            cv2.imwrite(crop_path, word_crop)


def resize_and_clean_words(crop_folder):
    if not os.path.exists(crop_folder):
        raise ValueError(f"The folder '{crop_folder}' does not exist.")

    image_files = [f for f in os.listdir(crop_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for img_file in image_files:
        img_path = os.path.join(crop_folder, img_file)
        image = cv2.imread(img_path)
        
        # Resize the image
        resized_image = resize_image(image)
        
        # Save the resized image (overwrite the original cropped image)
        cv2.imwrite(img_path, resized_image)


In [10]:
#CVL
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [11]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)

In [3]:
#IAM
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [8]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)